In [ ]:
import scipy 
print(scipy.__version__)


In [ ]:
import pandas as pd

# Replace 'file_path' with the path to your .feather file
file_path = "skin_atlas/GWAS/gsmap_test/mel6767_all_raw/generate_ldscore/SNP_gene_pair//SNP_gene_pair_chr16.feather"
df = pd.read_feather(file_path)

# Display the first few rows
print(df.head())


In [ ]:
df[df['gene_name'].str.contains("IL34", na=False)]

In [ ]:
#! pip install anndata==0.10.8

import anndata
print(anndata.__version__)
import pickle
import scanpy as sc
import numpy as np
import pandas as pd

In [ ]:
print(np.__version__)


In [ ]:
# Load the AnnData object from the pickle file
with open('/scratch/project/stseq/Levi/SkinCancerAtlas/cci_results/21031_Mel_stlearn.pkl', 'rb') as f:
    mel = pickle.load(f)
mel

In [ ]:
mel.obs

In [ ]:
mel.obsm['lr_sig_scores']

In [ ]:
keys_to_remove = ['lrfeatures', 'lr_summary', 'lr_cci_cell_type', 'lr_cci_raw_cell_type', 'per_lr_cci_cell_type', 'per_lr_cci_pvals_cell_type', 'per_lr_cci_raw_cell_type']  # Replace 'key_name' with the actual key you want to remove
for key in keys_to_remove:
    if key in mel.uns:
        del mel.uns[key]

# List all keys in adata.obsm
obsm_keys_to_remove=['spot_neighbours', 'spot_neigh_bcs', 'lr_scores', 'p_vals', 'p_adjs', '-log10(p_adjs)', 'lr_sig_scores']
for key in obsm_keys_to_remove:
    if key in mel.obsm:
        del mel.obsm[key]
    else:
        print(f"Key '{key}' not found in mel.obsm.")

mel

In [ ]:
mel.write('/QRISdata/Q1851/Quan/GWAS/Melanoma_SumStat/CM_sumstats/mel_test.h5ad')


In [ ]:
import numpy as np
#mel.layers['count'] = mel.X.todense()
#mel.layers['count'] = np.asarray(mel.X)

# If mel.X is a sparse matrix, convert it to a dense numpy.ndarray
if hasattr(mel.X, "todense"):  # Check if X is sparse
    mel.layers['count'] = np.asarray(mel.X.todense())  # Convert sparse to dense
else:
    mel.layers['count'] = np.asarray(mel.X)  # X is already a dense array

mel

In [ ]:
mel.write('mel_test.h5ad')


In [ ]:
mel.layers['count']

In [ ]:
mel2 = sc.read_10x_h5("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium25_Melanoma/FFPE_C1s3_6474_21031/outs/filtered_feature_bc_matrix.h5")

In [ ]:
mel2.X.todense()

In [ ]:
mel2

In [ ]:
mel

In [ ]:
# Check if rows (cells) match between mel and mel2
# mel2 should have more cells than mel (3901 vs. 2618)
# You can either take a subset of mel2 or align them if necessary

# Here, we'll assume they align for this example
assert mel2.shape[1] == mel.shape[1], "Number of features (columns) should match."

# If the row indices (cells) match, add mel2.X to mel.layers['count']
# Add mel2.X to mel.layers['count']
mel.layers['count'] = mel.layers['count'] + mel2.X[:mel.shape[0], :]

# Check the result
print(mel.layers['count'])


In [ ]:
# Check if 'count' layer exists in mel
if 'count' not in mel.layers:
    # If it doesn't exist, create the 'count' layer
    mel.layers['count'] = mel.X.copy()  # Assuming you want to start with the existing matrix in 'X'

# Now add mel2.X to mel.layers['count']
# Ensure both matrices are of compatible shapes (same number of features)
assert mel2.shape[1] == mel.shape[1], "Number of features (columns) should match."

# If mel2 has more cells, subset mel2.X to match the number of cells in mel
mel.layers['count'] = mel.layers['count'] + mel2.X[:mel.shape[0], :]

# Check the result
print(mel.layers['count'])


In [ ]:
mel

In [ ]:
mel.write('skin_atlas/GWAS/gsmap_test/mel_test.h5ad')

In [ ]:
mel.obs['cell_type'].value_counts()

In [ ]:
# 66487
# Load the AnnData object from the pickle file
with open('/scratch/project/stseq/Levi/SkinCancerAtlas/cci_results/66487_Mel_stlearn.pkl', 'rb') as f:
    mel = pickle.load(f)
mel
mel2 = sc.read_10x_h5("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium25_Melanoma/FFPE_A1s1_66487_48974/outs/filtered_feature_bc_matrix.h5")


keys_to_remove = ['lrfeatures', 'lr_summary', 'lr_cci_cell_type', 'lr_cci_raw_cell_type', 'per_lr_cci_cell_type', 'per_lr_cci_pvals_cell_type', 'per_lr_cci_raw_cell_type']  # Replace 'key_name' with the actual key you want to remove
for key in keys_to_remove:
    if key in mel.uns:
        del mel.uns[key]

# List all keys in adata.obsm
obsm_keys_to_remove=['spot_neighbours', 'spot_neigh_bcs', 'lr_scores', 'p_vals', 'p_adjs', '-log10(p_adjs)', 'lr_sig_scores']
for key in obsm_keys_to_remove:
    if key in mel.obsm:
        del mel.obsm[key]
    else:
        print(f"Key '{key}' not found in mel.obsm.")

print(mel)

# Step 1: Find matching barcodes between mel and mel2
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include these matching barcodes
mel2 = mel2[matching_indices, :]

# Step 3: Verify the shape of the subsetted mel2
print(f"Shape of the subsetted mel2: {mel2.shape}")

if 'count' not in mel.layers:
    # If it doesn't exist, create the 'count' layer
    mel.layers['count'] = mel.X.copy()  # Assuming you want to start with the existing matrix in 'X'


# Step 1: Ensure matching cells between mel and mel2 (based on row names or barcodes)
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include matching cells
mel2_subset = mel2[matching_indices, :]

# Step 3: Convert matching_indices to integer indices for mel (if needed)
matching_indices_int = mel.obs.index.get_indexer(matching_indices)

# Step 4: Add raw counts from mel2.X to mel.layers['counts']
# Use the raw counts from mel2.X (ensure it's in integer format if necessary)
mel.layers['count'][matching_indices_int, :] = mel2_subset.X.astype(int)

# Step 5: Verify the update
print(f"Updated mel.layers['counts'] shape: {mel.layers['count'].shape}")
print(mel.layers['count'].todense())

mel.write('skin_atlas/GWAS/gsmap_test/mel66487_raw.h5ad')

In [ ]:
# 6767
# Load the AnnData object from the pickle file
with open('/scratch/project/stseq/Levi/SkinCancerAtlas/cci_results/6767_Mel_stlearn.pkl', 'rb') as f:
    mel = pickle.load(f)
mel
mel2 = sc.read_10x_h5("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium25_Melanoma/FFPE_C1s3_6474_21031/outs/filtered_feature_bc_matrix.h5")


keys_to_remove = ['lrfeatures', 'lr_summary', 'lr_cci_cell_type', 'lr_cci_raw_cell_type', 'per_lr_cci_cell_type', 'per_lr_cci_pvals_cell_type', 'per_lr_cci_raw_cell_type']  # Replace 'key_name' with the actual key you want to remove
for key in keys_to_remove:
    if key in mel.uns:
        del mel.uns[key]

# List all keys in adata.obsm
obsm_keys_to_remove=['spot_neighbours', 'spot_neigh_bcs', 'lr_scores', 'p_vals', 'p_adjs', '-log10(p_adjs)', 'lr_sig_scores']
for key in obsm_keys_to_remove:
    if key in mel.obsm:
        del mel.obsm[key]
    else:
        print(f"Key '{key}' not found in mel.obsm.")

print(mel)

# Step 1: Find matching barcodes between mel and mel2
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include these matching barcodes
mel2 = mel2[matching_indices, :]

# Step 3: Verify the shape of the subsetted mel2
print(f"Shape of the subsetted mel2: {mel2.shape}")

if 'count' not in mel.layers:
    # If it doesn't exist, create the 'count' layer
    mel.layers['count'] = mel.X.copy()  # Assuming you want to start with the existing matrix in 'X'


# Step 1: Ensure matching cells between mel and mel2 (based on row names or barcodes)
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include matching cells
mel2_subset = mel2[matching_indices, :]

# Step 3: Convert matching_indices to integer indices for mel (if needed)
matching_indices_int = mel.obs.index.get_indexer(matching_indices)

# Step 4: Add raw counts from mel2.X to mel.layers['counts']
# Use the raw counts from mel2.X (ensure it's in integer format if necessary)
mel.layers['count'][matching_indices_int, :] = mel2_subset.X.astype(int)

# Step 5: Verify the update
print(f"Updated mel.layers['counts'] shape: {mel.layers['count'].shape}")
print(mel.layers['count'].todense())

mel.write('skin_atlas/GWAS/gsmap_test/mel6767_raw.h5ad')

In [ ]:
# 48974
# Load the AnnData object from the pickle file
with open('/scratch/project/stseq/Levi/SkinCancerAtlas/cci_results/48974_Mel_stlearn.pkl', 'rb') as f:
    mel = pickle.load(f)
mel
mel2 = sc.read_10x_h5("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium25_Melanoma/FFPE_A1s1_66487_48974/outs/filtered_feature_bc_matrix.h5")

In [ ]:
keys_to_remove = ['lrfeatures', 'lr_summary', 'lr_cci_cell_type', 'lr_cci_raw_cell_type', 'per_lr_cci_cell_type', 'per_lr_cci_pvals_cell_type', 'per_lr_cci_raw_cell_type']  # Replace 'key_name' with the actual key you want to remove
for key in keys_to_remove:
    if key in mel.uns:
        del mel.uns[key]

# List all keys in adata.obsm
obsm_keys_to_remove=['spot_neighbours', 'spot_neigh_bcs', 'lr_scores', 'p_vals', 'p_adjs', '-log10(p_adjs)', 'lr_sig_scores']
for key in obsm_keys_to_remove:
    if key in mel.obsm:
        del mel.obsm[key]
    else:
        print(f"Key '{key}' not found in mel.obsm.")

mel

In [ ]:
# Step 1: Find matching barcodes between mel and mel2
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include these matching barcodes
mel2 = mel2[matching_indices, :]

# Step 3: Verify the shape of the subsetted mel2
print(f"Shape of the subsetted mel2: {mel2.shape}")


In [ ]:
if 'count' not in mel.layers:
    # If it doesn't exist, create the 'count' layer
    mel.layers['count'] = mel.X.copy()  # Assuming you want to start with the existing matrix in 'X'


# Step 1: Ensure matching cells between mel and mel2 (based on row names or barcodes)
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include matching cells
mel2_subset = mel2[matching_indices, :]

# Step 3: Convert matching_indices to integer indices for mel (if needed)
matching_indices_int = mel.obs.index.get_indexer(matching_indices)

# Step 4: Add raw counts from mel2.X to mel.layers['counts']
# Use the raw counts from mel2.X (ensure it's in integer format if necessary)
mel.layers['count'][matching_indices_int, :] = mel2_subset.X.astype(int)

# Step 5: Verify the update
print(f"Updated mel.layers['counts'] shape: {mel.layers['count'].shape}")


In [ ]:
mel.layers['count'].todense()

In [ ]:
# Check if 'count' layer exists in mel
if 'count' not in mel.layers:
    # If it doesn't exist, create the 'count' layer
    mel.layers['count'] = mel.X.copy()  # Assuming you want to start with the existing matrix in 'X'

# Now add mel2.X to mel.layers['count']
# Ensure both matrices are of compatible shapes (same number of features)
assert mel2.shape[1] == mel.shape[1], "Number of features (columns) should match."

# If mel2 has more cells, subset mel2.X to match the number of cells in mel
mel.layers['count'] = mel.layers['count'] + mel2.X[:mel.shape[0], :]
mel.obs['cell_type'].value_counts()

In [ ]:
mel_old=anndata.read_h5ad('skin_atlas/GWAS/gsmap_test/mel48974.h5ad')
mel_old.layers['count'].todense()

In [ ]:
mel.write('skin_atlas/GWAS/gsmap_test/mel48974_raw.h5ad')

In [ ]:
sc.pl.spatial(mel,color=['PMEL','DROSHA','PBRM1'], size=2, vmax=1)

In [ ]:
import anndata
mel=anndata.read_h5ad('skin_atlas/GWAS/gsmap_test/mel48974_raw.h5ad')
sc.pl.spatial(mel,color=['IL34','CSF1R','LTB','LTBR'], size=1.5, vmax=1, cmap="RdYlBu_r")

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

def calculate_gene_specificity(adata, gene, groupby='cell_type', normalize=True):
    """
    Calculate gene specificity score for a given gene from an AnnData object.
    
    Parameters:
    - adata: AnnData object containing single-cell data.
    - gene: str, gene name to calculate specificity for.
    - groupby: str, column in `adata.obs` to group by (e.g., cell type or cluster).
    - normalize: bool, whether to normalize specificity scores to range [0, 1].
    
    Returns:
    - specificity_scores: pd.DataFrame with specificity scores for each group.
    """
    if gene not in adata.var_names:
        raise ValueError(f"Gene {gene} not found in adata.var_names.")

    # Group cells by the specified column and calculate mean expression per group
    grouped = adata.to_df()[gene].groupby(adata.obs[groupby]).mean()
    mean_expression = grouped.values
    total_expression = mean_expression.sum()

    # Calculate specificity score for each group
    specificity_scores = mean_expression / total_expression

    # Normalize specificity scores if requested
    if normalize:
        specificity_scores = (specificity_scores - specificity_scores.min()) / (
            specificity_scores.max() - specificity_scores.min()
        )
    
    # Create a DataFrame for easy visualization
    specificity_df = pd.DataFrame({
        groupby: grouped.index,
        'specificity_score': specificity_scores
    }).sort_values(by='specificity_score', ascending=False)
    
    return specificity_df

def plot_gene_specificity(specificity_df, gene, groupby='cell_type'):
    """
    Plot the gene specificity scores.
    
    Parameters:
    - specificity_df: pd.DataFrame with specificity scores (from calculate_gene_specificity).
    - gene: str, gene name for title.
    - groupby: str, grouping column used in specificity calculation.
    """
    plt.figure(figsize=(10, 6))
    plt.barh(specificity_df[groupby], specificity_df['specificity_score'], color='skyblue')
    plt.xlabel('Gene Specificity Score')
    plt.ylabel(groupby.capitalize())
    plt.title(f'Gene Specificity Score for {gene}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

# Example Usage
# Load AnnData object
adata = mel

# Define the gene and grouping
gene_of_interest = "IL34"  # Replace with your gene
grouping_column = "cell_type"  # Replace with your grouping column (e.g., "cell_type", "cluster")

# Calculate specificity scores
specificity_df = calculate_gene_specificity(adata, gene_of_interest, groupby=grouping_column)

# Plot specificity scores
plot_gene_specificity(specificity_df, gene_of_interest, groupby=grouping_column)


In [ ]:
def add_gene_specificity_to_obs(adata, gene, groupby='cell_type', normalize=True):
    """
    Add Gene Specificity Score (GSS) for a given gene to adata.obs.
    
    Parameters:
    - adata: AnnData object containing single-cell data.
    - gene: str, gene name to calculate specificity for.
    - groupby: str, column in `adata.obs` to group by (e.g., cell type or cluster).
    - normalize: bool, whether to normalize specificity scores to range [0, 1].
    
    Returns:
    - Updated adata with a new column `gene_GSS` in `adata.obs`.
    """
    if gene not in adata.var_names:
        raise ValueError(f"Gene {gene} not found in adata.var_names.")

    # Group cells by the specified column and calculate mean expression per group
    grouped = adata.to_df()[gene].groupby(adata.obs[groupby]).mean()
    mean_expression = grouped.values
    total_expression = mean_expression.sum()

    # Calculate specificity score for each group
    specificity_scores = mean_expression / total_expression

    # Normalize specificity scores if requested
    if normalize:
        specificity_scores = (specificity_scores - specificity_scores.min()) / (
            specificity_scores.max() - specificity_scores.min()
        )

    # Create a mapping of group -> GSS
    specificity_map = dict(zip(grouped.index, specificity_scores))

    # Map GSS to each cell based on its group
    adata.obs[f'{gene}_GSS'] = adata.obs[groupby].map(specificity_map)

    return adata

# Example Usage
# Load AnnData object
adata = mel

# Define the gene and grouping
gene_of_interest = "IL34"  # Replace with your gene
grouping_column = "cell_type"  # Replace with your grouping column (e.g., "cell_type", "cluster")

# Add GSS to adata.obs
adata = add_gene_specificity_to_obs(adata, gene_of_interest, groupby=grouping_column)

# Verify the added column
print(adata.obs[[grouping_column, f'{gene_of_interest}_GSS']].head())
sc.pl.spatial(mel,color=['IL34_GSS'], size=1.5, vmax=2, cmap="RdYlBu_r")

In [ ]:
import numpy as np

def calculate_continuous_gss(adata, gene, normalize=True):
    """
    Calculate continuous Gene Specificity Score (GSS) for a given gene and add it to adata.obs.
    
    Parameters:
    - adata: AnnData object containing single-cell data.
    - gene: str, gene name to calculate specificity for.
    - normalize: bool, whether to normalize GSS values to range [0, 1].
    
    Returns:
    - Updated adata with a new column `gene_GSS` in `adata.obs`.
    """
    if gene not in adata.var_names:
        raise ValueError(f"Gene {gene} not found in adata.var_names.")
    
    # Extract expression of the gene across all cells
    gene_expression = adata[:, gene].X.toarray().flatten() if hasattr(adata[:, gene].X, "toarray") else adata[:, gene].X.flatten()
    
    # Compute total expression across all cells
    total_expression = np.sum(gene_expression)
    if total_expression == 0:
        raise ValueError(f"Gene {gene} has zero total expression across all cells.")
    
    # Compute GSS for each cell
    gss = gene_expression / total_expression
    
    # Normalize to range [0, 1] if requested
    if normalize:
        gss = (gss - gss.min()) / (gss.max() - gss.min()) if gss.max() != gss.min() else gss
    
    # Add GSS as a new column in adata.obs
    adata.obs[f'{gene}_GSS'] = gss
    
    return adata

# Example Usage
# Load AnnData object
import scanpy as sc
adata = mel

# Define the gene of interest
gene_of_interest = "IL34"  # Replace with your gene name

# Add continuous GSS to adata.obs
adata = calculate_continuous_gss(adata, gene_of_interest)

# Verify the added column
print(adata.obs[[f'{gene_of_interest}_GSS']].head())
sc.pl.spatial(mel,color=['IL34_GSS'], size=1.5, cmap="RdYlBu_r")

In [ ]:
import numpy as np
from scipy.stats import rankdata, gmean, norm

def calculate_gss_by_cell_type(adata, gene, bandwidth=1):
    """
    Calculate Gene Specificity Score (GSS) for a given gene, using cell types as microdomains.

    Parameters:
    - adata: AnnData object containing gene expression data.
    - gene: str, gene name to calculate GSS for.
    - bandwidth: float, bandwidth for the Gaussian kernel.

    Returns:
    - Updated adata.obs with GSS values added as a new column.
    """
    if gene not in adata.var_names:
        raise ValueError(f"Gene {gene} not found in adata.var_names.")
    
    if "cell_type" not in adata.obs:
        raise ValueError("No `cell_type` column found in `adata.obs`. Add cell type annotations first.")
    
    # Extract gene expression and cell type information
    gene_expression = (
        adata[:, gene].X.toarray().flatten() if hasattr(adata[:, gene].X, "toarray") else adata[:, gene].X.flatten()
    )
    cell_types = adata.obs["cell_type"]

    # Rank gene expression values across all cells
    gene_ranks = rankdata(gene_expression, method="average")

    # Compute geometric mean of ranks across all cells
    global_gmean = gmean(gene_ranks) if np.all(gene_ranks > 0) else 0

    # Initialize GSS values
    gss_values = np.zeros_like(gene_expression, dtype=np.float64)

    # Group by cell type to calculate domain-specific geometric means
    unique_cell_types = cell_types.unique()
    for cell_type in unique_cell_types:
        domain_mask = cell_types == cell_type  # Boolean mask for cells in this domain
        domain_ranks = gene_ranks[domain_mask]

        # Compute geometric mean for the domain
        domain_gmean = gmean(domain_ranks) if np.all(domain_ranks > 0) else 0

        if global_gmean > 0:
            # Calculate specificity score for the domain
            specificity_score = (domain_gmean * gene_ranks[domain_mask]) ** (1 / (len(domain_ranks) + 1)) / global_gmean

            # Threshold checks: Set to 0 if proportion condition fails
            domain_expression_sum = np.sum(gene_expression[domain_mask])
            total_expression_sum = np.sum(gene_expression)

            if np.any(specificity_score < 0) or (domain_expression_sum / total_expression_sum) < 1:
                specificity_score = 0

            # Project onto Gaussian kernel
            gss_values[domain_mask] = norm.pdf(specificity_score, scale=bandwidth)
        else:
            gss_values[domain_mask] = 0

    # Add GSS values to adata.obs
    adata.obs[f'{gene}_GSS'] = gss_values

    return adata


# Load AnnData object
adata = mel

# Ensure cell type information is available in adata.obs["cell_type"]
# Example: Assign cell type annotations if not already present
# adata.obs["cell_type"] = cell_type_annotations
gene_of_interest = "CSF1R"  # Replace with your gene name

# Calculate GSS for a specific gene
adata = calculate_gss_by_cell_type(adata, gene_of_interest)

# Verify the results
print(adata.obs[[f'{gene_of_interest}_GSS', "cell_type"]].head())



# Add continuous GSS to adata.obs
# Verify the added column
sc.pl.spatial(mel,color=['CSF1R_GSS'], size=1.5,cmap="RdYlBu_r")

In [ ]:
import anndata
mel=anndata.read_h5ad('skin_atlas/GWAS/gsmap_test/B18_SCC_raw.h5ad')
sc.pl.spatial(mel,color=['THBS2','ITGA6'], size=1.5, vmax=2, cmap="RdYlBu_r")

### mel21031

In [ ]:
with open('/scratch/project/stseq/Levi/SkinCancerAtlas/cci_results/6767_Mel_stlearn.pkl', 'rb') as f:
    mel = pickle.load(f)
mel
mel2 = sc.read_10x_h5("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium25_Melanoma/FFPE_C1s3_6474_21031/outs/filtered_feature_bc_matrix.h5")


In [ ]:
keys_to_remove = ['lrfeatures', 'lr_summary', 'lr_cci_cell_type', 'lr_cci_raw_cell_type', 'per_lr_cci_cell_type', 'per_lr_cci_pvals_cell_type', 'per_lr_cci_raw_cell_type']  # Replace 'key_name' with the actual key you want to remove
for key in keys_to_remove:
    if key in mel.uns:
        del mel.uns[key]

# List all keys in adata.obsm
obsm_keys_to_remove=['spot_neighbours', 'spot_neigh_bcs', 'lr_scores', 'p_vals', 'p_adjs', '-log10(p_adjs)', 'lr_sig_scores']
for key in obsm_keys_to_remove:
    if key in mel.obsm:
        del mel.obsm[key]
    else:
        print(f"Key '{key}' not found in mel.obsm.")

mel

In [ ]:
# Check if 'count' layer exists in mel
if 'count' not in mel.layers:
    # If it doesn't exist, create the 'count' layer
    mel.layers['count'] = mel.X.copy()  # Assuming you want to start with the existing matrix in 'X'

# Now add mel2.X to mel.layers['count']
# Ensure both matrices are of compatible shapes (same number of features)
assert mel2.shape[1] == mel.shape[1], "Number of features (columns) should match."

# If mel2 has more cells, subset mel2.X to match the number of cells in mel
mel.layers['count'] = mel.layers['count'] + mel2.X[:mel.shape[0], :]
mel.obs['cell_type'].value_counts()


In [ ]:
mel.write('skin_atlas/GWAS/gsmap_test/mel6767.h5ad')

In [ ]:
import pandas as pd

# Load the .feather file
df = pd.read_feather("skin_atlas/GWAS/gsmap_test/mel48974_all_raw/latent_to_gene/mel48974_all_raw_gene_marker_score.feather")

# Display the first few rows
df


In [ ]:
df

## SCC BCC

In [ ]:
# B18_SCC
# Load the AnnData object from the pickle file
with open('/scratch/project/stseq/Levi/SkinCancerAtlas/cci_results/B18_SCC_stlearn.pkl', 'rb') as f:
    mel = pickle.load(f)
mel
mel2 = sc.read_10x_h5("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium9_BCC_SCC/Visium9_B1/B1/outs/filtered_feature_bc_matrix.h5")


keys_to_remove = ['lrfeatures', 'lr_summary', 'lr_cci_cell_type', 'lr_cci_raw_cell_type', 'per_lr_cci_cell_type', 'per_lr_cci_pvals_cell_type', 'per_lr_cci_raw_cell_type']  # Replace 'key_name' with the actual key you want to remove
for key in keys_to_remove:
    if key in mel.uns:
        del mel.uns[key]

# List all keys in adata.obsm
obsm_keys_to_remove=['spot_neighbours', 'spot_neigh_bcs', 'lr_scores', 'p_vals', 'p_adjs', '-log10(p_adjs)', 'lr_sig_scores']
for key in obsm_keys_to_remove:
    if key in mel.obsm:
        del mel.obsm[key]
    else:
        print(f"Key '{key}' not found in mel.obsm.")

print(mel)

# Step 1: Find matching barcodes between mel and mel2
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include these matching barcodes
mel2 = mel2[matching_indices, :]

# Step 3: Verify the shape of the subsetted mel2
print(f"Shape of the subsetted mel2: {mel2.shape}")

if 'count' not in mel.layers:
    # If it doesn't exist, create the 'count' layer
    mel.layers['count'] = mel.X.copy()  # Assuming you want to start with the existing matrix in 'X'


# Step 1: Ensure matching cells between mel and mel2 (based on row names or barcodes)
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include matching cells
mel2_subset = mel2[matching_indices, :]

# Step 3: Convert matching_indices to integer indices for mel (if needed)
matching_indices_int = mel.obs.index.get_indexer(matching_indices)

# Step 4: Add raw counts from mel2.X to mel.layers['counts']
# Use the raw counts from mel2.X (ensure it's in integer format if necessary)
mel.layers['count'][matching_indices_int, :] = mel2_subset.X.astype(int)

# Step 5: Verify the update
print(f"Updated mel.layers['counts'] shape: {mel.layers['count'].shape}")
print(mel.layers['count'].todense())

mel.write('skin_atlas/GWAS/gsmap_test/B18_SCC_raw.h5ad')

In [ ]:
# B18_SCC
# Load the AnnData object from the pickle file
with open('/scratch/project/stseq/Levi/SkinCancerAtlas/cci_results/F21_BCC_stlearn.pkl', 'rb') as f:
    mel = pickle.load(f)
mel
mel2 = sc.read_10x_h5("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium9_BCC_SCC/Visium9_D1/D1/outs/filtered_feature_bc_matrix.h5")


keys_to_remove = ['lrfeatures', 'lr_summary', 'lr_cci_cell_type', 'lr_cci_raw_cell_type', 'per_lr_cci_cell_type', 'per_lr_cci_pvals_cell_type', 'per_lr_cci_raw_cell_type']  # Replace 'key_name' with the actual key you want to remove
for key in keys_to_remove:
    if key in mel.uns:
        del mel.uns[key]

# List all keys in adata.obsm
obsm_keys_to_remove=['spot_neighbours', 'spot_neigh_bcs', 'lr_scores', 'p_vals', 'p_adjs', '-log10(p_adjs)', 'lr_sig_scores']
for key in obsm_keys_to_remove:
    if key in mel.obsm:
        del mel.obsm[key]
    else:
        print(f"Key '{key}' not found in mel.obsm.")

print(mel)

# Step 1: Find matching barcodes between mel and mel2
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include these matching barcodes
mel2 = mel2[matching_indices, :]

# Step 3: Verify the shape of the subsetted mel2
print(f"Shape of the subsetted mel2: {mel2.shape}")

if 'count' not in mel.layers:
    # If it doesn't exist, create the 'count' layer
    mel.layers['count'] = mel.X.copy()  # Assuming you want to start with the existing matrix in 'X'


# Step 1: Ensure matching cells between mel and mel2 (based on row names or barcodes)
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include matching cells
mel2_subset = mel2[matching_indices, :]

# Step 3: Convert matching_indices to integer indices for mel (if needed)
matching_indices_int = mel.obs.index.get_indexer(matching_indices)

# Step 4: Add raw counts from mel2.X to mel.layers['counts']
# Use the raw counts from mel2.X (ensure it's in integer format if necessary)
mel.layers['count'][matching_indices_int, :] = mel2_subset.X.astype(int)

# Step 5: Verify the update
print(f"Updated mel.layers['counts'] shape: {mel.layers['count'].shape}")
print(mel.layers['count'].todense())

mel.write('skin_atlas/GWAS/gsmap_test/F21_BCC_raw.h5ad')



In [ ]:
mel

# BCC

In [ ]:
# B18_SCC
# Load the AnnData object from the pickle file
with open('/scratch/project/stseq/Levi/SkinCancerAtlas/cci_results/F21_BCC_stlearn.pkl', 'rb') as f:
    mel = pickle.load(f)
mel
mel2 = sc.read_10x_h5("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium9_BCC_SCC/Visium9_D1/D1/outs/filtered_feature_bc_matrix.h5")


keys_to_remove = ['lrfeatures', 'lr_summary', 'lr_cci_cell_type', 'lr_cci_raw_cell_type', 'per_lr_cci_cell_type', 'per_lr_cci_pvals_cell_type', 'per_lr_cci_raw_cell_type']  # Replace 'key_name' with the actual key you want to remove
for key in keys_to_remove:
    if key in mel.uns:
        del mel.uns[key]

# List all keys in adata.obsm
obsm_keys_to_remove=['spot_neighbours', 'spot_neigh_bcs', 'lr_scores', 'p_vals', 'p_adjs', '-log10(p_adjs)', 'lr_sig_scores']
for key in obsm_keys_to_remove:
    if key in mel.obsm:
        del mel.obsm[key]
    else:
        print(f"Key '{key}' not found in mel.obsm.")

print(mel)

# Step 1: Find matching barcodes between mel and mel2
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include these matching barcodes
mel2 = mel2[matching_indices, :]

# Step 3: Verify the shape of the subsetted mel2
print(f"Shape of the subsetted mel2: {mel2.shape}")

if 'count' not in mel.layers:
    # If it doesn't exist, create the 'count' layer
    mel.layers['count'] = mel.X.copy()  # Assuming you want to start with the existing matrix in 'X'


# Step 1: Ensure matching cells between mel and mel2 (based on row names or barcodes)
matching_indices = mel.obs.index.intersection(mel2.obs.index)

# Step 2: Subset mel2 to only include matching cells
mel2_subset = mel2[matching_indices, :]

# Step 3: Convert matching_indices to integer indices for mel (if needed)
matching_indices_int = mel.obs.index.get_indexer(matching_indices)

# Step 4: Add raw counts from mel2.X to mel.layers['counts']
# Use the raw counts from mel2.X (ensure it's in integer format if necessary)
mel.layers['count'][matching_indices_int, :] = mel2_subset.X.astype(int)

# Step 5: Verify the update
print(f"Updated mel.layers['counts'] shape: {mel.layers['count'].shape}")
print(mel.layers['count'].todense())

mel.write('skin_atlas/GWAS/gsmap_test/F21_BCC_raw.h5ad')



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [ ]:
# Load the data
data = pd.read_csv('skin_atlas/GWAS/gsmap_test/LR_for_plot', sep='\t', header=0)


# Plot Setup
plt.figure(figsize=(10, 8))

# Create a scatter plot with Ligand Cell Type vs Receptor Cell Type
sns.scatterplot(
    x='Ligand Cell Type', 
    y='Receptor Cell Type', 
    hue='Sample', 
    style='Sample', 
    data=data, 
    s=200
)

# Add connection lines between ligand and receptor cell types
for _, row in data.iterrows():
    plt.plot(
        [row['Ligand Cell Type'], row['Receptor Cell Type']], 
        [row['Ligand'], row['Receptor']],
        color='gray', 
        linestyle='dotted'
    )

# Customize the plot
plt.title('Ligand-Receptor Interactions Across Cell Types', fontsize=16)
plt.xlabel('Ligand Cell Type', fontsize=12)
plt.ylabel('Receptor Cell Type', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(title='Sample', fontsize=10)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Build a graph from the data
G = nx.DiGraph()

for _, row in data.iterrows():
    G.add_edge(f"{row['Ligand']} ({row['Ligand Cell Type']})",
               f"{row['Receptor']} ({row['Receptor Cell Type']})",
               label=row['Sample'])

# Draw the graph
pos = nx.spring_layout(G)
plt.figure(figsize=(12, 8))
nx.draw(
    G, pos, with_labels=True, node_color="lightblue", 
    edge_color="gray", font_size=5, node_size=300
)
plt.title("Ligand-Receptor Interaction Network", fontsize=16)
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



# Create a pivot table for the heatmap
heatmap_data = data.groupby(['Sample', 'Ligand Cell Type', 'Receptor Cell Type']).size().reset_index(name='Count')
heatmap_pivot = heatmap_data.pivot_table(
    index='Ligand Cell Type', 
    columns=['Sample', 'Receptor Cell Type'], 
    values='Count', 
    fill_value=0
)

# Show the pivot table
#print(heatmap_pivot)

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_pivot, cmap='Blues', annot=True, fmt='d', linewidths=0.5)
plt.title("Ligand-Receptor Interactions by Sample and Cell Types", fontsize=16)
plt.ylabel("Ligand Cell Type")
plt.xlabel("Sample and Receptor Cell Type")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Example DataFrame
data = pd.read_csv('skin_atlas/GWAS/gsmap_test/LR_for_plot', sep='\t', header=0)

# Combine Ligand and Receptor into a single axis
data['Ligand-Receptor'] = data['Ligand'] + " → " + data['Receptor']

# Group and pivot data for heatmap
heatmap_data = data.groupby(['Ligand-Receptor', 'Ligand Cell Type', 'Receptor Cell Type', 'Sample']).size().reset_index(name='Count')
heatmap_pivot = heatmap_data.pivot_table(
    index='Ligand-Receptor', 
    columns=['Ligand Cell Type', 'Receptor Cell Type'], 
    values='Count', 
    fill_value=0
)

# Map samples to colors
sample_colors = {
    'B18_SCC': 'red',
    'E15_SCC':'pink',
    'P30_SCC':'orange',
    'P13_SCC':'yellow',
    'B18_BCC':'brown',
    'mel6767_LRs': 'blue',
    'mel48974_LRs': 'green',
    'mel66487_LRs': 'violet',
}
row_colors = [sample_colors[sample] for sample in data['Sample']]

# Create heatmap with sample annotations
sns.clustermap(
    heatmap_pivot,
    cmap='coolwarm',
    row_cluster=False,
    col_cluster=False,
    figsize=(12, 10),
    row_colors=row_colors,
    linewidths=0.5,
    annot=True,
    fmt='d'
)

# Customize the plot
plt.title("Ligand-Receptor Interactions Heatmap with Sample Annotations", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Example DataFrame
data = pd.read_csv('skin_atlas/GWAS/gsmap_test/LR_for_plot', sep='\t', header=0)


# Combine Ligand and Receptor into a single axis
data['Ligand-Receptor'] = data['Ligand'] + " → " + data['Receptor']

# Group and pivot data for heatmap
heatmap_data = data.groupby(['Ligand-Receptor', 'Ligand Cell Type', 'Receptor Cell Type', 'Sample']).size().reset_index(name='Count')
heatmap_pivot = heatmap_data.pivot_table(
    index='Ligand-Receptor', 
    columns=['Ligand Cell Type', 'Receptor Cell Type'], 
    values='Count', 
    fill_value=0
)

sample_colors = {
    'B18_SCC': 'red',
    'E15_SCC':'pink',
    'P30_SCC':'orange',
    'P13_SCC':'yellow',
    'B18_BCC':'brown',
    'mel6767_LRs': 'blue',
    'mel48974_LRs': 'green',
    'mel66487_LRs': 'violet',
}
row_colors = [sample_colors[sample] for sample in data['Sample']]

# Create the clustermap
clustermap = sns.clustermap(
    heatmap_pivot,
    cmap='coolwarm',
    row_cluster=False,
    col_cluster=False,
    figsize=(12, 8),
    row_colors=row_colors,
    linewidths=0.5,
    annot=True,
    fmt='d'
)

# Add legend for sample annotations
handles = [mpatches.Patch(color=color, label=sample) for sample, color in sample_colors.items()]
clustermap.ax_heatmap.legend(
    handles=handles,     loc='lower left',
    bbox_to_anchor=(-1.05, -1),  # Position legend outside the heatmap
    title="Samples"
)

# Customize the plot
plt.title("Ligand-Receptor Interactions Heatmap with Sample Annotations", fontsize=16, pad=20)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


# Combine Ligand and Receptor into a single axis
data['Ligand-Receptor'] = data['Ligand'] + " → " + data['Receptor']

# Group and pivot data for heatmap
heatmap_data = data.groupby(['Ligand-Receptor', 'Ligand Cell Type', 'Receptor Cell Type', 'Sample']).size().reset_index(name='Count')
heatmap_pivot = heatmap_data.pivot_table(
    index='Ligand-Receptor', 
    columns=['Ligand Cell Type', 'Receptor Cell Type'], 
    values='Count', 
    fill_value=0
)

# Ensure all samples are included in row_colors
unique_samples = data['Sample'].unique()
sample_colors = {
    sample: color for sample, color in zip(unique_samples, sns.color_palette('Set1', len(unique_samples)))
}
row_sample_map = {row: data.loc[data['Ligand-Receptor'] == row, 'Sample'].iloc[0] 
                  if row in data['Ligand-Receptor'].values else 'Unknown' 
                  for row in heatmap_pivot.index}
row_colors = [sample_colors[row_sample_map[row]] for row in heatmap_pivot.index]

# Create the clustermap
clustermap = sns.clustermap(
    heatmap_pivot,
    cmap='coolwarm',
    row_cluster=False,
    col_cluster=False,
    figsize=(12, 8),
    row_colors=row_colors,
    linewidths=0.5,
    annot=True,
    fmt='d'
)

# Add legend for sample annotations
handles = [mpatches.Patch(color=color, label=sample) for sample, color in sample_colors.items()]
clustermap.ax_heatmap.legend(
    handles=handles,
    loc='upper left',
    bbox_to_anchor=(1.05, 1),  # Position legend outside the heatmap
    title="Samples"
)

# Customize the plot
plt.title("Ligand-Receptor Interactions Heatmap with Sample Annotations", fontsize=16, pad=20)
plt.tight_layout()
plt.show()


# GSS plots

In [ ]:
#b18=anndata.read_h5ad("skin_atlas/GWAS/gsmap_test/B18_SCC_raw.h5ad")
b18 = st.Read10X("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium25_Melanoma/FFPE_A1s1_66487_48974/outs")
B18_GSS=pd.read_csv("skin_atlas/GWAS/gsmap_test/mel48974_all_raw/report/mel/gsMap_plot/mel48974_all_raw_mel_gsMap_plot.csv",
                    header=0, index_col=0)
b18
sc.pp.pca(b18)
sc.pp.neighbors(b18)
sc.tl.umap(b18)
# Add 'logp' from df to b18.obs['GSS'] for matching barcodes
b18.obs["GSS"] = np.nan  # Initialize with NaN
matching_barcodes = b18.obs.index.isin(B18_GSS.index)

# Update 'GSS' for matching barcodes
b18.obs.loc[matching_barcodes, "GSS"] = B18_GSS.loc[b18.obs.index[matching_barcodes], "logp"]

if "GSS" in b18.var_names:
    print("Removing GSS from .var_names...")
    b18 = b18[:, b18.var_names != "GSS"].copy()
if "GSS" in b18.obs.columns:
    print("GSS is in .obs. Ready for plotting.")
else:
    raise ValueError("GSS is missing from .obs.")


sc.pl.spatial(
    b18,
    color="GSS",        # Use GSS from .obs
    cmap="RdYlBu_r",     # Colormap for visualization
    size=1.5,           # Adjust spot size
    alpha_img=0.5 , vmax=5      # Set background transparency
)


In [ ]:
#b18=anndata.read_h5ad("skin_atlas/GWAS/gsmap_test/B18_SCC_raw.h5ad")
b18 = st.Read10X("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium25_Melanoma/FFPE_A1s1_66487_48974/outs")
B18_GSS=pd.read_csv("skin_atlas/GWAS/gsmap_test/mel66487_all_raw/report/mel/gsMap_plot/mel66487_all_raw_mel_gsMap_plot.csv",
                    header=0, index_col=0)
b18
sc.pp.pca(b18)
sc.pp.neighbors(b18)
sc.tl.umap(b18)
# Add 'logp' from df to b18.obs['GSS'] for matching barcodes
b18.obs["GSS"] = np.nan  # Initialize with NaN
matching_barcodes = b18.obs.index.isin(B18_GSS.index)

# Update 'GSS' for matching barcodes
b18.obs.loc[matching_barcodes, "GSS"] = B18_GSS.loc[b18.obs.index[matching_barcodes], "logp"]

if "GSS" in b18.var_names:
    print("Removing GSS from .var_names...")
    b18 = b18[:, b18.var_names != "GSS"].copy()
if "GSS" in b18.obs.columns:
    print("GSS is in .obs. Ready for plotting.")
else:
    raise ValueError("GSS is missing from .obs.")


sc.pl.spatial(
    b18,
    color="GSS",        # Use GSS from .obs
    cmap="RdYlBu_r",     # Colormap for visualization
    size=1.5,           # Adjust spot size
    alpha_img=0.5 , vmax=5      # Set background transparency
)


In [ ]:
#b18=anndata.read_h5ad("skin_atlas/GWAS/gsmap_test/B18_SCC_raw.h5ad")
b18 = st.Read10X("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium25_Melanoma/FFPE_C1s3_6474_21031/outs")
B18_GSS=pd.read_csv("skin_atlas/GWAS/gsmap_test/mel6767_all_raw/report/mel/gsMap_plot/mel6767_all_sig_raw_mel_gsMap_plot.csv",
                    header=0, index_col=0)
b18
sc.pp.pca(b18)
sc.pp.neighbors(b18)
sc.tl.umap(b18)
# Add 'logp' from df to b18.obs['GSS'] for matching barcodes
b18.obs["GSS"] = np.nan  # Initialize with NaN
matching_barcodes = b18.obs.index.isin(B18_GSS.index)

# Update 'GSS' for matching barcodes
b18.obs.loc[matching_barcodes, "GSS"] = B18_GSS.loc[b18.obs.index[matching_barcodes], "logp"]

if "GSS" in b18.var_names:
    print("Removing GSS from .var_names...")
    b18 = b18[:, b18.var_names != "GSS"].copy()
if "GSS" in b18.obs.columns:
    print("GSS is in .obs. Ready for plotting.")
else:
    raise ValueError("GSS is missing from .obs.")


sc.pl.spatial(
    b18,
    color="GSS",        # Use GSS from .obs
    cmap="RdYlBu_r",     # Colormap for visualization
    size=1.5,           # Adjust spot size
    alpha_img=0.5 , vmax=5      # Set background transparency
)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv("skin_atlas/GWAS/gsmap_test/mel6767_all_raw/report/mel/mel6767_all_sig_raw_mel_Gene_Diagnostic_Info.csv", header=0)
# Extract the 'all_Cancer' data from adata.obs
all_cancer_values = df['PCC']

# Plot a histogram using seaborn for better visualization
plt.figure(figsize=(8, 5))
sns.histplot(all_cancer_values, kde=False, bins=30, color='blue', edgecolor='black')

# Add titles and labels
plt.title('Histogram of all_Cancer', fontsize=14)
plt.xlabel('all_Cancer', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

# Calculate the 90th percentile
percentile_75 = np.percentile(all_cancer_values, 75)
percentile_80 = np.percentile(all_cancer_values, 80)
percentile_90 = np.percentile(all_cancer_values, 90)
percentile_95 = np.percentile(all_cancer_values, 95)
percentile_99 = np.percentile(all_cancer_values, 99)
percentile_995 = np.percentile(all_cancer_values, 99.5)


# Print the result
print(f"75th Percentile of 'all_Cancer': {percentile_75}")
print(f"80th Percentile of 'all_Cancer': {percentile_80}")
print(f"90th Percentile of 'all_Cancer': {percentile_90}")
print(f"95th Percentile of 'all_Cancer': {percentile_95}")
print(f"99th Percentile of 'all_Cancer': {percentile_99}")


# SCC

In [ ]:
#b18=anndata.read_h5ad("skin_atlas/GWAS/gsmap_test/B18_SCC_raw.h5ad")
b18 = st.Read10X("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium9_BCC_SCC/Visium9_B1/B1/outs/")
B18_GSS=pd.read_csv("skin_atlas/GWAS/gsmap_test/B18_SCC_all_eff_n_raw/report/scc_eff_n/gsMap_plot/B18_SCC_all_eff_n_raw_scc_eff_n_gsMap_plot.csv",
                    header=0, index_col=0)
b18
sc.pp.pca(b18)
sc.pp.neighbors(b18)
sc.tl.umap(b18)
# Add 'logp' from df to b18.obs['GSS'] for matching barcodes
b18.obs["GSS"] = np.nan  # Initialize with NaN
matching_barcodes = b18.obs.index.isin(B18_GSS.index)

# Update 'GSS' for matching barcodes
b18.obs.loc[matching_barcodes, "GSS"] = B18_GSS.loc[b18.obs.index[matching_barcodes], "logp"]

if "GSS" in b18.var_names:
    print("Removing GSS from .var_names...")
    b18 = b18[:, b18.var_names != "GSS"].copy()
if "GSS" in b18.obs.columns:
    print("GSS is in .obs. Ready for plotting.")
else:
    raise ValueError("GSS is missing from .obs.")


sc.pl.spatial(
    b18,
    color="GSS",        # Use GSS from .obs
    cmap="RdYlBu_r",     # Colormap for visualization
    size=1.5,           # Adjust spot size
    alpha_img=0.5 , vmax=5      # Set background transparency
)


In [ ]:

#b18=anndata.read_h5ad("skin_atlas/GWAS/gsmap_test/B18_SCC_raw.h5ad")
b18 = st.Read10X("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium9_BCC_SCC/Visium9_B1/B1/outs/")
B18_GSS=pd.read_csv("skin_atlas/GWAS/gsmap_test/E15_SCC_all_eff_n_raw/report/scc_eff_n/gsMap_plot/E15_SCC_all_eff_n_raw_scc_eff_n_gsMap_plot.csv",
                    header=0, index_col=0)
b18
sc.pp.pca(b18)
sc.pp.neighbors(b18)
sc.tl.umap(b18)
# Add 'logp' from df to b18.obs['GSS'] for matching barcodes
b18.obs["GSS"] = np.nan  # Initialize with NaN
matching_barcodes = b18.obs.index.isin(B18_GSS.index)

# Update 'GSS' for matching barcodes
b18.obs.loc[matching_barcodes, "GSS"] = B18_GSS.loc[b18.obs.index[matching_barcodes], "logp"]

if "GSS" in b18.var_names:
    print("Removing GSS from .var_names...")
    b18 = b18[:, b18.var_names != "GSS"].copy()
if "GSS" in b18.obs.columns:
    print("GSS is in .obs. Ready for plotting.")
else:
    raise ValueError("GSS is missing from .obs.")


sc.pl.spatial(
    b18,
    color="GSS",        # Use GSS from .obs
    cmap="RdYlBu_r",     # Colormap for visualization
    size=1.5,           # Adjust spot size
    alpha_img=0.5 , vmax=5      # Set background transparency
)


# BCC

In [ ]:
#b18=anndata.read_h5ad("skin_atlas/GWAS/gsmap_test/B18_SCC_raw.h5ad")
b18 = st.Read10X("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium32_BCC_SCC/A1_B18BCC/outs/")
B18_GSS=pd.read_csv("skin_atlas/GWAS/gsmap_test/B18_BCC_all_eff_n_raw/report/bcc/gsMap_plot/B18_BCC_all_eff_n_raw_bcc_gsMap_plot.csv",
                    header=0, index_col=0)
b18
sc.pp.pca(b18)
sc.pp.neighbors(b18)
sc.tl.umap(b18)
# Add 'logp' from df to b18.obs['GSS'] for matching barcodes
b18.obs["GSS"] = np.nan  # Initialize with NaN
matching_barcodes = b18.obs.index.isin(B18_GSS.index)

# Update 'GSS' for matching barcodes
b18.obs.loc[matching_barcodes, "GSS"] = B18_GSS.loc[b18.obs.index[matching_barcodes], "logp"]

if "GSS" in b18.var_names:
    print("Removing GSS from .var_names...")
    b18 = b18[:, b18.var_names != "GSS"].copy()
if "GSS" in b18.obs.columns:
    print("GSS is in .obs. Ready for plotting.")
else:
    raise ValueError("GSS is missing from .obs.")


sc.pl.spatial(
    b18,
    color="GSS",        # Use GSS from .obs
    cmap="RdYlBu_r",     # Colormap for visualization
    size=1.5,           # Adjust spot size
    alpha_img=0.5 , vmax=5      # Set background transparency
)


In [ ]:
#b18=anndata.read_h5ad("skin_atlas/GWAS/gsmap_test/B18_SCC_raw.h5ad")
b18 = st.Read10X("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium9_BCC_SCC/Visium9_D1/D1/outs/")
B18_GSS=pd.read_csv("skin_atlas/GWAS/gsmap_test/F21_BCC_all_eff_n_raw/report/bcc/gsMap_plot/F21_BCC_all_eff_n_raw_bcc_gsMap_plot.csv",
                    header=0, index_col=0)
b18
sc.pp.pca(b18)
sc.pp.neighbors(b18)
sc.tl.umap(b18)
# Add 'logp' from df to b18.obs['GSS'] for matching barcodes
b18.obs["GSS"] = np.nan  # Initialize with NaN
matching_barcodes = b18.obs.index.isin(B18_GSS.index)

# Update 'GSS' for matching barcodes
b18.obs.loc[matching_barcodes, "GSS"] = B18_GSS.loc[b18.obs.index[matching_barcodes], "logp"]

if "GSS" in b18.var_names:
    print("Removing GSS from .var_names...")
    b18 = b18[:, b18.var_names != "GSS"].copy()
if "GSS" in b18.obs.columns:
    print("GSS is in .obs. Ready for plotting.")
else:
    raise ValueError("GSS is missing from .obs.")


sc.pl.spatial(
    b18,
    color="GSS",        # Use GSS from .obs
    cmap="RdYlBu_r",     # Colormap for visualization
    size=1.5,           # Adjust spot size
    alpha_img=0.5 , vmax=5      # Set background transparency
)


In [ ]:
def generate_GSS_distribution(config: DiagnosisConfig):
    """Generate GSS distribution plots."""
    # logger.info('Loading ST data...')
    # adata = sc.read_h5ad(config.hdf5_with_latent_path)
    mk_score = pd.read_feather(config.mkscore_feather_path).set_index('HUMAN_GENE_SYM').T

    plot_genes = config.selected_genes or load_gene_diagnostic_info(config).Gene.iloc[:config.top_corr_genes].tolist()
    if config.selected_genes is not None:
        logger.info(f'Generating GSS & Expression distribution plot for selected genes: {plot_genes}...')
    else:
        logger.info(f'Generating GSS & Expression distribution plot for top {config.top_corr_genes} correlated genes...')

    if config.customize_fig:
        pixel_width, pixel_height, point_size = config.fig_width, config.fig_height, config.point_size
    else:
        (pixel_width, pixel_height), point_size = estimate_point_size_for_plot(adata.obsm['spatial'])
    sub_fig_save_dir = config.get_GSS_plot_dir(config.trait_name)

def generate_and_save_plots(adata, mk_score, expression_series, selected_gene, point_size, pixel_width, pixel_height,
                            sub_fig_save_dir, sample_name, annotation):
    """Generate and save the plots."""
    select_gene_expression_with_space_coord = load_st_coord(adata, expression_series, annotation)
    sub_fig_1 = draw_scatter(select_gene_expression_with_space_coord, title=f'{selected_gene} (Expression)',
                             annotation='annotation', color_by='Expression', point_size=point_size, width=pixel_width,
                             height=pixel_height)
    save_plot(sub_fig_1, sub_fig_save_dir, sample_name, selected_gene, 'Expression')

    select_gene_GSS_with_space_coord = load_st_coord(adata, mk_score[selected_gene].rename('GSS'), annotation)
    sub_fig_2 = draw_scatter(select_gene_GSS_with_space_coord, title=f'{selected_gene} (GSS)', annotation='annotation',
                             color_by='GSS', point_size=point_size, width=pixel_width, height=pixel_height)
    save_plot(sub_fig_2, sub_fig_save_dir, sample_name, selected_gene, 'GSS')

    # combined_fig = make_subplots(rows=1, cols=2,
    #                              subplot_titles=(f'{selected_gene} (Expression)', f'{selected_gene} (GSS)'))
    # for trace in sub_fig_1.data:
    #     combined_fig.add_trace(trace, row=1, col=1)
    # for trace in sub_fig_2.data:
    #     combined_fig.add_trace(trace, row=1, col=2)
    #

In [ ]:
selected_gene="IL34"
adata = st.Read10X("/QRISdata/Q2051/SCC_Paper/figures/Fig2_VisiumNanostring/previous_versions/Fig3_VisiumNanostring_celltypes/working_data/Visium25_Melanoma/FFPE_A1s1_66487_48974/outs")
expression_series = pd.Series(adata[:, selected_gene].X.toarray().flatten(), index=adata.obs.index,name='Expression')
threshold = np.quantile(expression_series,0.9999)
expression_series[expression_series > threshold] = threshold
generate_and_save_plots(adata, mk_score, expression_series, selected_gene)

In [ ]:
import anndata 
d=anndata.read_h5ad("/scratch/project/stseq/Prakrithi/skin_atlas/GWAS/gsmap_test/mel48974_all_raw/find_latent_representations/mel48974_all_raw_add_latent.h5ad")
d

In [ ]:
d.obsm['latent_GVAE']

In [ ]:
# Check for 'GSS' in adata.obs and adata.var_names
if "GSS" in adata.var_names:
    print("Removing GSS from var_names to avoid conflict.")
    adata.var.drop("GSS", inplace=True)

# Rename 'GSS' in adata.obs to avoid potential conflicts
if "GSS" in adata.obs.columns:
    print("Renaming GSS in obs to GSS_score to avoid conflicts.")
    adata.obs.rename(columns={"GSS": "GSS_score"}, inplace=True)

# Ensure the renamed column exists and is properly set
if "GSS_score" not in adata.obs.columns:
    print("Adding mock GSS_score values for testing.")
    adata.obs["GSS_score"] = np.random.rand(adata.n_obs)  # Replace with real GSS values

# Ensure spatial coordinates exist in obsm
if "spatial" not in adata.obsm.keys():
    print("Adding mock spatial coordinates for testing.")
    adata.obsm["spatial"] = np.random.rand(adata.n_obs, 2)  # Replace with actual spatial coordinates

# Gene of interest (optional, customize based on your use case)
gene_of_interest = "IL34"  # Replace with the actual gene of interest

# Plot GSS score for spatial data
sc.pl.spatial(
    adata,
    color="GSS_score",  # Column renamed above
    cmap="RdYlBu_r",  # Choose a color map
    title=f"GSS for {gene_of_interest}",
    size=1.5,  # Adjust size for better visualization
    show=True,
)

In [ ]:
# Check for 'GSS' in adata.obs and adata.var_names
if "GSS" in adata.var_names:
    print("Removing GSS from var_names to avoid conflict.")
    adata.var.drop("GSS", inplace=True)

# Rename 'GSS' in adata.obs to avoid potential conflicts
if "GSS" in adata.obs.columns:
    print("Renaming GSS in obs to GSS_score to avoid conflicts.")
    adata.obs.rename(columns={"GSS": "GSS_score"}, inplace=True)

# Ensure the renamed column exists and is properly set
if "GSS_score" not in adata.obs.columns:
    print("Adding mock GSS_score values for testing.")
    adata.obs["GSS_score"] = np.random.rand(adata.n_obs)  # Replace with real GSS values

# Ensure spatial coordinates exist in obsm
if "spatial" not in adata.obsm.keys():
    print("Adding mock spatial coordinates for testing.")
    adata.obsm["spatial"] = np.random.rand(adata.n_obs, 2)  # Replace with actual spatial coordinates

# Gene of interest (optional, customize based on your use case)
gene_of_interest = "CSF1R"  # Replace with the actual gene of interest

# Plot GSS score for spatial data
sc.pl.spatial(
    adata,
    color="GSS_score",  # Column renamed above
    cmap="RdYlBu_r",  # Choose a color map
    title=f"GSS for {gene_of_interest}",
    size=1.5,  # Adjust size for better visualization
    show=True, vmax=1
)

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.stats import gmean
from scipy.sparse import issparse

# Load raw count data
adata = sc.read_h5ad("skin_atlas/GWAS/gsmap_test/mel48974_raw.h5ad")




# Ensure micro-domain (cell type) column exists in .obs
if "cell_type" not in adata.obs:
    raise ValueError("Micro-domain ('cell_type') information is missing in adata.obs!")

# Step 1: Rank genes in each spot (higher values = higher ranks)
# Convert sparse matrix to dense matrix
dense_data = adata.X.toarray() if sc.sparse.issparse(adata.X) else adata.X

# Rank genes in each spot (axis=1). Higher values get higher ranks.
ranked_data = np.argsort(np.argsort(dense_data, axis=1), axis=1) + 1  # 1-based rank
adata.layers["ranked"] = ranked_data

# Step 2: Group by micro-domains (cell types)
micro_domains = adata.obs["cell_type"]

# Initialize dictionary to store specificity scores
specificity_scores = pd.DataFrame(index=adata.obs_names, columns=adata.var_names)

for cell_type in micro_domains.unique():
    # Extract spots belonging to this micro-domain
    micro_domain_spots = adata[micro_domains == cell_type, :]

    # Convert to dense matrix if sparse
    dense_micro_data = (
        micro_domain_spots.layers["ranked"].toarray()
        if sc.sparse.issparse(micro_domain_spots.layers["ranked"])
        else micro_domain_spots.layers["ranked"]
    )

    # Compute geometric mean of ranks within the micro-domain
    geometric_mean_micro = gmean(dense_micro_data, axis=0)

    # Compute geometric mean of ranks across all spots
    geometric_mean_all = gmean(ranked_data, axis=0)

    # Compute F_ig for each gene
    F_ig = (
        geometric_mean_micro / geometric_mean_all
    ) ** (1 / (len(micro_domain_spots) + 1))  # Formula from screenshot

    # Filter outliers
    F_ig[F_ig < 0] = 0

    # Compute S_ig using Gaussian kernel transformation
    S_ig = np.exp(F_ig**2 - 1)

    # Store specificity scores for this micro-domain
    specificity_scores.loc[micro_domains == cell_type, :] = S_ig

# Add specificity scores to adata.obs
adata.obs = adata.obs.join(specificity_scores)

# Save results


In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

# Define a function to calculate Gene Specificity Score (GSS)
def calculate_gss(adata, gene, cell_type_column):
    """
    Calculate GSS for a specific gene based on raw counts and cell type.
    
    Parameters:
        adata: AnnData object with raw counts in adata.X.
        gene: The gene of interest (string).
        cell_type_column: Column in adata.obs representing cell types.

    Returns:
        DataFrame with cell types and their corresponding GSS values.
    """
    # Extract raw counts and ensure gene is present
    if gene not in adata.var_names:
        raise ValueError(f"Gene {gene} not found in the dataset.")
    
    gene_expression = adata[:, gene].X.toarray().flatten() if hasattr(adata[:, gene].X, "toarray") else adata[:, gene].X

    # Add gene expression to adata.obs for processing
    adata.obs["gene_expression"] = gene_expression

    # Group by cell type and calculate mean expression within each group
    grouped = adata.obs.groupby(cell_type_column)["gene_expression"].mean()

    # Calculate global mean rank of the gene
    global_rank = pd.Series(gene_expression).rank().mean()

    # Calculate GSS: mean rank in each group divided by global mean rank
    gss = grouped.rank() / global_rank

    return pd.DataFrame({"CellType": grouped.index, "GSS": gss.values})

# Load your AnnData object
# Example: adata = sc.read_h5ad("your_adata_file.h5ad")

# Parameters
gene_of_interest = "IL34"
cell_type_column = "cell_type"  # Adjust based on your adata.obs column name

# Calculate GSS
gss_df = calculate_gss(adata, gene_of_interest, cell_type_column)

# Plot GSS
plt.figure(figsize=(10, 6))
sns.barplot(data=gss_df, x="CellType", y="GSS", palette="viridis")
plt.title(f"Gene Specificity Score (GSS) for {gene_of_interest}", fontsize=14)
plt.xlabel("Cell Type", fontsize=12)
plt.ylabel("GSS", fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
gss_df

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
import matplotlib.pyplot as plt

adata = sc.read_h5ad("skin_atlas/GWAS/gsmap_test/mel48974_raw.h5ad")
# Define parameters
gene_of_interest = "IL34"  # Replace with your gene of interest
num_hvgs = 3000  # Number of highly variable genes to select
k_neighbors = 10  # Number of nearest neighbors in the spatial graph

# Step 1: Preprocess the data
adata.layers["raw_counts"] = adata.layers["count"]  # Store counts as raw counts
sc.pp.normalize_total(adata, target_sum=1e6, layer="raw_counts")  # Normalize raw counts
sc.pp.log1p(adata, layer="raw_counts")  # Log-transform
sc.pp.highly_variable_genes(adata, n_top_genes=num_hvgs, layer="raw_counts")  # Select top HVGs
#adata = adata[:, adata.var["highly_variable"]]  # Filter to HVGs

# Standardize the gene expression matrix
scaler = StandardScaler(with_mean=False)  # Avoid centering for sparse matrices
adata.layers["standardized"] = scaler.fit_transform(adata.layers["raw_counts"])

# Step 2: Build the spatial graph
# Ensure spatial coordinates are present in `adata.obsm`
if "spatial" not in adata.obsm:
    raise ValueError("Spatial coordinates are missing from `adata.obsm['spatial']`.")

spatial_coords = adata.obsm["spatial"]
adj_matrix = kneighbors_graph(spatial_coords, n_neighbors=k_neighbors, mode="connectivity", include_self=False)

# Step 3: Compute embedding matrix using cosine similarity
embeddings = []
for i, spot_vec in enumerate(spatial_coords):
    distances = np.linalg.norm(spatial_coords - spot_vec, axis=1)
    neighbors = np.argsort(distances)[:k_neighbors]
    embedding = adata.layers["standardized"][neighbors].mean(axis=0)  # Average over neighbors
    embeddings.append(embedding)

embedding_matrix = np.vstack(embeddings)

# Step 4: Calculate Gene Specificity Score (GSS)
gss_values = []
gene_expression = adata.layers["standardized"][:, adata.var_names.get_loc(gene_of_interest)].toarray().flatten() if hasattr(adata.layers["standardized"], "toarray") else adata.layers["standardized"][:, adata.var_names.get_loc(gene_of_interest)]
ranks = np.argsort(np.argsort(gene_expression)) + 1  # Rank within all spots

for j in range(adata.n_obs):
    neighbors = adj_matrix[j].nonzero()[1]
    local_ranks = ranks[neighbors]
    global_mean_rank = ranks.mean()
    local_mean_rank = np.exp(np.log(local_ranks).mean())
    gss = local_mean_rank / global_mean_rank
    if gss < 1:
        gss_values.append(0)
    else:
        gss_values.append(gss)

# Add GSS values for the gene of interest to adata.obs
adata.obs[f"{gene_of_interest}_GSS"] = gss_values

# Step 5: Plot GSS for the gene of interest
plt.figure(figsize=(10, 6))
sc.pl.spatial(
    adata,
    color=f"{gene_of_interest}_GSS",
    size=2,
    cmap="RdYlBu_r",
    title=f"GSS for {gene_of_interest}", vmax=2, alpha_img=0
)
plt.show()


In [ ]:
sc.pl.spatial(
    adata,
    color="IL34",
    size=1.5,
    cmap="RdYlBu_r", vmax=2, alpha_img=0
)

# Xenium 

In [ ]:
m = anndata.read_h5ad("/QRISdata/Q2051/Levi/SCC/Xenium_Melanoma_0017002__23346-10SP__20231129__074525.h5ad")
m

In [ ]:
sc.pl.spatial(
    m,
    color="pred_celltype_ucdbase"
)

In [ ]:
m.layers['counts']

In [ ]:
xenium_dir <- "/QRISdata/Q2051/Feng/SkinCancer/process-data/Xenium_data"
patient_folder <- 'output-XETG00114__0017002__6475-07FC__20231129__074525'
patient <- '6475-07FC'
xenium.obj <- LoadXenium(file.path(xenium_dir,patient_folder), fov = "fov")
all_metadata <- read.csv("/QRISdata/Q2051/Feng/SkinCancer/clean-data/all_metadata.csv")
metadata <- all_metadata[all_metadata$patient_cancer==paste0(patient,'_Mel'),]
row.names(metadata) <- metadata$cell_id
xenium.obj <- AddMetaData(xenium.obj,metadata=metadata[c("cell_type")])